In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
# Function to read stock data from an Excel file
def get_stock_data_from_excel(file_path):
    all_stock_data = pd.read_excel(file_path, sheet_name=None)
    stock_data = {}

    for sheet_name, data in all_stock_data.items():
        # Ensure 'Date' is treated as a datetime column and set as index
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)
        stock_data[sheet_name] = data

    return stock_data

In [ ]:
# Function to calculate EMA for a given stock's dataframe
def calculate_ema(data, period):
    return data['Close'].ewm(span=period, adjust=False).mean()

In [ ]:
# Function to calculate Stochastic Oscillator
def calculate_stochastic(data, period_k, period_d, period_smooth):
    lowest_low = data['Low'].rolling(window=period_k).min()
    highest_high = data['High'].rolling(window=period_k).max()
    k_percent = (data['Close'] - lowest_low) * 100 / (highest_high - lowest_low)
    k_percent_smooth = k_percent.rolling(window=period_smooth).mean()
    d_percent = k_percent_smooth.rolling(window=period_d).mean()
    return k_percent_smooth, d_percent

In [ ]:
# Function to calculate ATR (Average True Range)
def calculate_atr(data, period):
    high_low = data['High'] - data['Low']
    high_close = np.abs(data['High'] - data['Close'].shift())
    low_close = np.abs(data['Low'] - data['Close'].shift())
    true_range = high_low.combine(high_close, max).combine(low_close, max)
    atr = true_range.rolling(window=period).mean()
    return atr

In [ ]:
# Function to calculate ADX (Average Directional Index)
def calculate_adx(df, period):
    # Store the original index
    original_index = df.index
    # Reset the index to ensure it starts from 0
    df = df.reset_index(drop=True)
    # Calculate High, Low, and Close price differences
    df['H-PH'] = df['High'] - df['High'].shift(1)
    df['PL-L'] = df['Low'].shift(1) - df['Low']
    df['TR'] = np.maximum(df['High'] - df['Low'], np.maximum(abs(df['High'] - df['Close'].shift(1)), abs(df['Low'] - df['Close'].shift(1))))

    # Calculate +DM and -DM
    df['+DM'] = np.where((df['H-PH'] > df['PL-L']) & (df['H-PH'] > 0), df['H-PH'], 0)
    df['-DM'] = np.where((df['PL-L'] > df['H-PH']) & (df['PL-L'] > 0), df['PL-L'], 0)

    # Initialize smoothed values
    df['Smooth +DM'] = np.nan
    df['Smooth -DM'] = np.nan
    df['Smooth TR'] = np.nan
    #df['Smooth TR'] = df['TR'].rolling(window=period).mean()

    # Apply the new smoothing formula
    smoothing_factor = 1 / period

    # Initialize smoothed values with the first period's sum
    df.loc[period-1, 'Smooth +DM'] = df['+DM'].iloc[:period].sum()
    df.loc[period-1, 'Smooth -DM'] = df['-DM'].iloc[:period].sum()
    df.loc[period-1, 'Smooth TR'] = df['TR'].iloc[:period].sum()

    # Apply Welles Wilder Smoothing for the rest of the periods
    #for i in range(period, len(df)):
        #df.loc[i, 'Smooth +DM'] = (df.loc[i-1, 'Smooth +DM'] * (period - 1) + df.loc[i, '+DM']) / period
        #df.loc[i, 'Smooth -DM'] = (df.loc[i-1, 'Smooth -DM'] * (period - 1) + df.loc[i, '-DM']) / period
        #df.loc[i, 'Smooth TR'] = (df.loc[i-1, 'Smooth TR'] * (period - 1) + df.loc[i, 'TR']) / period

    # New Smoothing Exponential Moving Average
    for i in range(period, len(df)):
        df.loc[i, 'Smooth +DM'] = df.loc[i, '+DM'] * smoothing_factor + df.loc[i-1, 'Smooth +DM'] * (1 - smoothing_factor)
        df.loc[i, 'Smooth -DM'] = df.loc[i, '-DM'] * smoothing_factor + df.loc[i-1, 'Smooth -DM'] * (1 - smoothing_factor)
        df.loc[i, 'Smooth TR'] = df.loc[i, 'TR'] * smoothing_factor + df.loc[i-1, 'Smooth TR'] * (1 - smoothing_factor)

    # Calculate +DI and -DI
    df['+DI'] = (df['Smooth +DM'] / df['Smooth TR']) * 100
    df['-DI'] = (df['Smooth -DM'] / df['Smooth TR']) * 100

    # Calculate DX
    df['DX'] = (abs(df['+DI'] - df['-DI']) / (df['+DI'] + df['-DI'])) * 100

    # Initialize ADX column
    df['ADX'] = np.nan
    df.loc[2 * period - 1, 'ADX'] = df['DX'].iloc[period-1:2*period-1].mean()  # First ADX value is the average of the first `period` DX values

    # Calculate ADX for the rest of the periods using Welles Wilder Smoothing
    #for i in range(2 * period, len(df)):
        #df.loc[i, 'ADX'] = (df.loc[i-1, 'ADX'] * (period - 1) + df.loc[i, 'DX']) / period

    #New Smoothing for ADX (Exponential Moving Average)
    for i in range(2 * period, len(df)):
        df.loc[i, 'ADX'] = df.loc[i, 'DX'] * smoothing_factor + df.loc[i-1, 'ADX'] * (1 - smoothing_factor)

    # Restore the original index
    df.index = original_index

    return df['ADX']

In [ ]:
# Function to create the SCRIP DataFrame for Bounce Trade
def create_scrip_dataframe(data, ema_periods, stochastic_periods, adx_period, atr_period):
    # Calculate EMAs, Stochastic Oscillator, ATR, and ADX
    data['EMA_8'] = calculate_ema(data, ema_periods['fast'])
    data['EMA_21'] = calculate_ema(data, ema_periods['mid1'])
    data['EMA_34'] = calculate_ema(data, ema_periods['mid2'])
    data['EMA_55'] = calculate_ema(data, ema_periods['mid3'])
    data['EMA_89'] = calculate_ema(data, ema_periods['slow'])

    data['%K'], data['%D'] = calculate_stochastic(data, stochastic_periods['k'], stochastic_periods['d'], stochastic_periods['smooth'])
    data['ATR'] = calculate_atr(data, atr_period)
    data['ADX'] = calculate_adx(data, adx_period)

    # Initialize 'Long/Short', 'Signal Number', 'Stop-Loss', 'Target', 'SL Hit', 'Target Hit' columns
    data['Long / Short'] = ''
    data['Signal Number'] = 0
    data['Stop-Loss'] = np.nan
    data['Target'] = np.nan
    data['SL Hit'] = ''
    data['Target Hit'] = ''

    signal_number = 1
    active_trade = False
    entry_price = 0

    update_sl_target_next = False  # Flag to trigger the update of SL and Target in the next row
    next_stop_loss = np.nan
    next_target = np.nan

    # Loop through each row and identify Long/Short signals
    for i in range(4, len(data)):  # Start from 4th index to access previous days
        # If there is an active trade, carry forward the trade details (Signal Number and Long/Short)
        if active_trade:
            data.at[data.index[i], 'Long / Short'] = data['Long / Short'].iloc[i-1]
            data.at[data.index[i], 'Signal Number'] = data['Signal Number'].iloc[i-1]
            data.at[data.index[i], 'Stop-Loss'] = data['Stop-Loss'].iloc[i-1]  # Carry forward the last known Stop-Loss
            data.at[data.index[i], 'Target'] = data['Target'].iloc[i-1]  # Carry forward the last known Target
            # Reset 'SL Hit' and 'Target Hit' to 'NO' initially
            data.at[data.index[i], 'SL Hit'] = 'NO'
            data.at[data.index[i], 'Target Hit'] = 'NO'

            # If the flag is set, update Stop-Loss and Target for the current row
            if update_sl_target_next:
                data.at[data.index[i], 'Stop-Loss'] = next_stop_loss
                data.at[data.index[i], 'Target'] = next_target
                update_sl_target_next = False  # Reset flag after update

        # Long Setup
        if (data['EMA_8'].iloc[i] > data['EMA_21'].iloc[i] > data['EMA_34'].iloc[i] >
            data['EMA_55'].iloc[i] > data['EMA_89'].iloc[i] and
            data['%D'].iloc[i] < 40 and data['ADX'].iloc[i] >= 20 and
            data['Close'].iloc[i] >= (data['EMA_21'].iloc[i] - data['ATR'].iloc[i]) and
            data['Close'].iloc[i] <= (data['EMA_21'].iloc[i] + data['ATR'].iloc[i])):

            min_close_index_label = data['Close'].iloc[i-4:i+1].idxmin()
            min_close_index_position = data.index.get_loc(min_close_index_label)

            if data['Close'].iloc[i+1] > data['High'].iloc[min_close_index_position]:
                if not active_trade:
                    # New Long Signal
                    data.at[data.index[i], 'Long / Short'] = 'Long'
                    data.at[data.index[i], 'Signal Number'] = signal_number
                    entry_price = data['Close'].iloc[i]
                    data.at[data.index[i], 'Stop-Loss'] = entry_price - data['ATR'].iloc[i]  # Initial stop-loss
                    data.at[data.index[i], 'Target'] = entry_price + data['ATR'].iloc[i]  # Initial target
                    signal_number += 1
                    active_trade = True

        # Short Setup
        elif (data['EMA_8'].iloc[i] < data['EMA_21'].iloc[i] < data['EMA_34'].iloc[i] <
              data['EMA_55'].iloc[i] < data['EMA_89'].iloc[i] and
              data['%D'].iloc[i] > 60 and data['ADX'].iloc[i] >= 20 and
              data['Close'].iloc[i] >= (data['EMA_21'].iloc[i] - data['ATR'].iloc[i]) and
              data['Close'].iloc[i] <= (data['EMA_21'].iloc[i] + data['ATR'].iloc[i])):

              max_close_index_label = data['Close'].iloc[i-4:i+1].idxmax()
              max_close_index_position = data.index.get_loc(max_close_index_label)

              if data['Close'].iloc[i+1] < data['Low'].iloc[max_close_index_position]:
                  if not active_trade:
                      # New Short Signal
                      data.at[data.index[i], 'Long / Short'] = 'Short'
                      data.at[data.index[i], 'Signal Number'] = signal_number
                      entry_price = data['Close'].iloc[i]
                      data.at[data.index[i], 'Stop-Loss'] = entry_price + data['ATR'].iloc[i]  # Initial stop-loss
                      data.at[data.index[i], 'Target'] = entry_price - data['ATR'].iloc[i]  # Initial target
                      signal_number += 1
                      active_trade = True

        # Check if Target Hit or Stop-Loss Hit
        if active_trade:
            if data['Long / Short'].iloc[i] == 'Long':
                # Check for Stop-Loss Hit
                if data['Close'].iloc[i] < data['Stop-Loss'].iloc[i]:
                    data.at[data.index[i], 'SL Hit'] = 'YES'
                    active_trade = False  # Exit trade

                # Check for Target Hit
                elif data['Close'].iloc[i] > data['Target'].iloc[i]:
                    data.at[data.index[i], 'Target Hit'] = 'YES'
                    # Set flag to update SL and Target in the next row
                    if i + 1 < len(data):  # Ensure the index is within bounds
                        next_stop_loss = data['Stop-Loss'].iloc[i] + data['ATR'].iloc[i+1]
                        next_target = data['Target'].iloc[i] + data['ATR'].iloc[i+1]
                        update_sl_target_next = True

            elif data['Long / Short'].iloc[i] == 'Short':
                # Check for Stop-Loss Hit
                if data['Close'].iloc[i] > data['Stop-Loss'].iloc[i]:
                    data.at[data.index[i], 'SL Hit'] = 'YES'
                    active_trade = False  # Exit trade

                # Check for Target Hit
                elif data['Close'].iloc[i] < data['Target'].iloc[i]:
                    data.at[data.index[i], 'Target Hit'] = 'YES'
                    # Set flag to update SL and Target in the next row
                    if i + 1 < len(data):  # Ensure the index is within bounds
                        next_stop_loss = data['Stop-Loss'].iloc[i] - data['ATR'].iloc[i+1]
                        next_target = data['Target'].iloc[i] - data['ATR'].iloc[i+1]
                        update_sl_target_next = True

    return data


In [ ]:
# Function to create the Trade_Results DataFrame collectively for all trades
def create_trade_results(scrip_df):
    trades = []
    grouped = scrip_df.groupby('Signal Number')

    for signal_number, group in grouped:
        # Entry is the day of the signal generation
        entry = group.iloc[0]

        # If it is the last signal, check if it has a valid exit
        if signal_number == scrip_df['Signal Number'].max():
            # If the signal ends before the data is over, use the actual exit
            if len(group) > 1:
                signal_exit_index = group.index[-1]
                exit = scrip_df.shift(-1).loc[signal_exit_index]  # Get the next day's data
            else:
                # If the signal does not have an exit, use the last available data as exit
                exit = scrip_df.iloc[-1]
        else:
            # Exit is the day after the signal ends for non-last signals
            signal_exit_index = group.index[-1]
            exit = scrip_df.shift(-1).loc[signal_exit_index]

        entry_index = scrip_df.index.get_loc(entry.name) + 1
        exit_index = scrip_df.index.get_loc(exit.name) + 2

        # Ensure the index is within bounds
        if entry_index < len(scrip_df):
            entry = scrip_df.iloc[entry_index]
            if exit_index >= len(scrip_df):
                exit = scrip_df.iloc[-1]  # Use the last available day as exit if beyond data
            else:
                exit = scrip_df.iloc[exit_index]
        else:
            continue

        entry_date = entry.name
        exit_date = exit.name

        entry_price = entry['Open']
        exit_price = exit['Open']
        days_in_trade = (exit_date - entry_date).days

        # Determine if the signal is a whipsaw signal
        whipsaw_signal_count = scrip_df[scrip_df['Signal Number'] == signal_number].shape[0]
        whipsaw_signal = 'Yes' if whipsaw_signal_count == 1 else 'No'

        # Determine the range excluding the entry date
        period_data = scrip_df[(scrip_df.index >= entry_date) & (scrip_df.index < exit_date)]

        # Calculate highest or lowest point based on the signal type
        highest_lowest_point = period_data['High'].max() if entry['Long / Short'] == 'Long' else period_data['Low'].min()
        highest_point_percent = ((highest_lowest_point - entry_price) / entry_price * 100) if entry['Long / Short'] == 'Long' else ((entry_price - highest_lowest_point) / entry_price * 100)

        # Determine if the highest point occurred on the opening day (excluding entry day)
        opening_highest = 'Yes' if highest_point_percent <= 0.0 else 'No'

        trades.append({
            'Signal Number': signal_number,
            'Type of Signal': entry['Long / Short'],
            'Entry Date': entry_date,
            'Exit Date': exit_date,
            'Entry Price': entry_price,
            'Exit Price': exit_price,
            'Days in Trade': days_in_trade,
            'Pure Signal P&L': 0.0,  # Placeholder, will be calculated separately
            'Pure Signal Won/Lost': '',  # Placeholder, will be calculated separately
            'Highest Point/Lowest Point': highest_lowest_point,
            'Highest Point %': highest_point_percent,
            'Opening = Highest': opening_highest,
            'Whipsaw Signal': whipsaw_signal,
            'Opening Equity': 0.0,  # Placeholder, will be calculated separately
            'Shares Bought': 0,  # Placeholder, will be calculated separately
            'Total Price Paid': 0.0,  # Placeholder, will be calculated separately
            'Total Price Got': 0.0,  # Placeholder, will be calculated separately
            'Transaction Cost': 0.0,  # Placeholder, will be calculated separately
            'Closing Equity': 0.0,  # Placeholder, will be calculated separately
        })

    trade_results_df = pd.DataFrame(trades)
    return trade_results_df


In [ ]:
# Function to calculate financial metrics for a given trade results DataFrame
def calculate_financial_metrics(trade_results_df, starting_equity):
    previous_equity = starting_equity
    for i, row in trade_results_df.iterrows():
        shares_bought = round(previous_equity / row['Entry Price'])
        total_price_paid = shares_bought * row['Entry Price']
        total_price_got = shares_bought * row['Exit Price']
        transaction_cost = total_price_paid * 0.002
        pnl = ((row['Exit Price'] - row['Entry Price']) * shares_bought) - transaction_cost if row['Type of Signal'] == 'Long' else ((row['Entry Price'] - row['Exit Price']) * shares_bought) - transaction_cost
        won_lost = 'Won' if pnl > 0 else 'Lost'
        closing_equity = previous_equity + pnl

        # Update the dataframe
        trade_results_df.at[i, 'Opening Equity'] = previous_equity
        trade_results_df.at[i, 'Shares Bought'] = shares_bought
        trade_results_df.at[i, 'Total Price Paid'] = total_price_paid
        trade_results_df.at[i, 'Total Price Got'] = total_price_got
        trade_results_df.at[i, 'Transaction Cost'] = transaction_cost
        trade_results_df.at[i, 'Pure Signal P&L'] = pnl
        trade_results_df.at[i, 'Pure Signal Won/Lost'] = won_lost
        trade_results_df.at[i, 'Closing Equity'] = closing_equity

        previous_equity = closing_equity

    return trade_results_df

In [ ]:
# Function to create the Analysis DataFrame for a given set of trades
def create_analysis_dataframe(trade_results_df, signal_type, stock_symbol, starting_equity):
    total_trades = len(trade_results_df)
    winning_trades = trade_results_df[trade_results_df['Pure Signal Won/Lost'] == 'Won']
    losing_trades = trade_results_df[trade_results_df['Pure Signal Won/Lost'] == 'Lost']
    winning_percentage = (len(winning_trades) / total_trades) * 100 if total_trades > 0 else 0
    losing_percentage = (len(losing_trades) / total_trades) * 100 if total_trades > 0 else 0
    num_whipsaws = trade_results_df[trade_results_df['Whipsaw Signal'] == 'Yes'].shape[0]
    num_opening_highest = trade_results_df[trade_results_df['Opening = Highest'] == 'Yes'].shape[0]
    thirtieth_percentile = trade_results_df['Highest Point %'].quantile(0.3) if total_trades > 0 else 0

    # Calculate CAGR (Compound Annual Growth Rate)
    if total_trades > 0:
        num_years = trade_results_df['Days in Trade'].sum() / 365
        ending_equity = trade_results_df.iloc[-1]['Closing Equity']
        cagr = (((ending_equity / starting_equity) ** (1 / num_years)) - 1) * 100 if num_years > 0 else 0
    else:
        cagr = 0

    analysis_data = {
        'Scrip Name': stock_symbol,
        #'Fast EMA': fast_ema,
        #'Slow EMA': slow_ema,
        '# Signals': total_trades,
        'Signal Type': signal_type,
        'CAGR- Return': cagr,
        'Winning %': winning_percentage,
        'Losing %': losing_percentage,
        '# Whipsaws': num_whipsaws,
        '# Opening = Highest': num_opening_highest,
        '30th Percentile': thirtieth_percentile
    }
    analysis_df = pd.DataFrame([analysis_data])
    return analysis_df

In [ ]:
# Define EMA, Stochastic, ADX, and ATR periods
ema_periods = {'fast': 8, 'mid1': 21, 'mid2': 34, 'mid3': 55, 'slow': 89}
stochastic_periods = {'k': 8, 'd': 3, 'smooth': 3}
adx_period = 13
atr_period = 14

In [ ]:
# Path to the Excel file containing stock data
file_path = '/content/drive/MyDrive/Data/DATA_FOR_TESTING_P2.xlsx'

In [ ]:
# Get stock data from Excel file
stock_data = get_stock_data_from_excel(file_path)

In [ ]:
#selected_stock = {'ABB','ICICIBANK','INFY','MARUTI','RELIANCE','SBIN','WIPRO'}
#selected_stock = {'ABB'}

In [ ]:
starting_equity = 100000  # Initial equity

In [ ]:
# Initialize lists to store all Long and Short results for all stocks
all_long_analysis_results = []
all_short_analysis_results = []

In [ ]:
for stock_name in stock_data.keys():
    print(f"Processing stock: {stock_name}")
    if stock_name in stock_data:
        data = stock_data[stock_name]

        # Generate Scrip Dataframe
        scrip_df = create_scrip_dataframe(data, ema_periods, stochastic_periods, adx_period, atr_period)

        # Generate Trade Results
        trade_results_df = create_trade_results(scrip_df)

        # Separate trade results into long and short
        long_trade_results_df = trade_results_df[trade_results_df['Type of Signal'] == 'Long'].reset_index(drop=True)
        short_trade_results_df = trade_results_df[trade_results_df['Type of Signal'] == 'Short'].reset_index(drop=True)

        # Calculate metrics separately for long and short trades
        long_trade_results_df = calculate_financial_metrics(long_trade_results_df, starting_equity)
        short_trade_results_df = calculate_financial_metrics(short_trade_results_df, starting_equity)

        # Generate analysis for long and short trades
        long_analysis_df = create_analysis_dataframe(long_trade_results_df, 'Long', stock_name, starting_equity)
        short_analysis_df = create_analysis_dataframe(short_trade_results_df, 'Short', stock_name, starting_equity)

        # Store the analysis results for all stocks
        all_long_analysis_results.append(long_analysis_df)
        all_short_analysis_results.append(short_analysis_df)

Processing stock: AARTIIND
Processing stock: ABB
Processing stock: ABBOTINDIA
Processing stock: ABCAPITAL
Processing stock: ABFRL
Processing stock: ACC
Processing stock: ADANIENT
Processing stock: ADANIPORTS
Processing stock: ALKEM
Processing stock: AMBUJACEM
Processing stock: APOLLOHOSP
Processing stock: APOLLOTYRE
Processing stock: ASHOKLEY
Processing stock: ASIANPAINT
Processing stock: ASTRAL
Processing stock: ATUL
Processing stock: AUBANK
Processing stock: AUROPHARMA
Processing stock: AXISBANK
Processing stock: BAJAJ-AUTO
Processing stock: BAJAJFINSV
Processing stock: BAJFINANCE
Processing stock: BALKRISIND
Processing stock: BALRAMCHIN
Processing stock: BANDHANBNK
Processing stock: BANKBARODA
Processing stock: BATAINDIA
Processing stock: BEL
Processing stock: BERGEPAINT
Processing stock: BHARATFORG
Processing stock: BHARTIARTL
Processing stock: BHEL
Processing stock: BIOCON
Processing stock: BOSCHLTD
Processing stock: BPCL
Processing stock: BRITANNIA
Processing stock: BSOFT
Process

In [ ]:
# Concatenate the analysis results for all stocks into two DataFrames
final_long_analysis_df = pd.concat(all_long_analysis_results, ignore_index=True)
final_short_analysis_df = pd.concat(all_short_analysis_results, ignore_index=True)

In [ ]:
# Save the analysis data to a new Excel file with each stock's data in its own sheet
output_file_path = '/content/drive/MyDrive/Analysis/Bounce Trade/All_Stocks_Analysis_Bounce_Trade.xlsx'

In [ ]:
with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='w') as writer:
    final_long_analysis_df.to_excel(writer, sheet_name='Analysis Long', index=False)
    final_short_analysis_df.to_excel(writer, sheet_name='Analysis Short', index=False)

In [ ]:
# Confirm file save
print(f"Data saved to {output_file_path}")

Data saved to /content/drive/MyDrive/Analysis/Bounce Trade/All_Stocks_Analysis_Bounce_Trade.xlsx
